# Out of fold evaluation for EfficientDet

Hi everyone!

My name is Alex Shonenkov, I am DL/NLP/CV/TS research engineer. Especially I am in Love with NLP & DL.

Recently I have created kernels for this competition:
- [WBF approach for ensemble](https://www.kaggle.com/shonenkov/wbf-approach-for-ensemble)
- [[Training] EfficientDet](https://www.kaggle.com/shonenkov/training-efficientdet)
- [[Inference] EfficientDet](https://www.kaggle.com/shonenkov/inference-efficientdet)

# Main Idea

People from ODS slack asked me about competition metrics on my validation split for effdet. So I have decided to publish notebook with calculation oof-score for my models and selection best threshold :) 

For this aims I have trained 5-folds using published [training kernel](https://www.kaggle.com/shonenkov/training-efficientdet). 

It is very simple notebook without any really good idea, but I hope It can help you evaluate effdet models in the future during research!
So, lets start! 

In [1]:
#!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
#!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [2]:
import sys
sys.path.insert(0, "../../efficientdet-pytorch")
sys.path.insert(0, "../../omegaconf")
sys.path.insert(0, "../weightedboxesfusion")

In [3]:
from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet
from sklearn.model_selection import StratifiedKFold

from metrics import calculate_final_score, predict_eval_set, eval_metrics

# Prepare Folds

In [4]:
DATA_DIR = '../../data/wheat'
marking = pd.read_csv(f'{DATA_DIR}/train.csv')

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = marking[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

/relevance2-nfs/chec/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [10]:
marking.head()

,image_id,width,height,source,x,y,w,h
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0


In [6]:
def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [7]:
TRAIN_ROOT_PATH = f'{DATA_DIR}/train'


def collate_fn(batch):
    return tuple(zip(*batch))


class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]

        image, boxes = self.load_image_and_boxes(index)

        # there is only one class
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
#                     target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes

In [8]:
    validation_dataset = DatasetRetriever(
        image_ids=df_folds[df_folds['fold'] == 0].index.values,
        marking=marking,
        transforms=get_valid_transforms(),
        test=True,
    )

    validation_loader = DataLoader(
        validation_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=2,
        drop_last=False,
        collate_fn=collate_fn
    )

# Load Models

In [8]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

#models = [
    #load_net('./ed5-mixup/best-fold-0.pth'),
    #load_net('./ed5-mixup/best-fold-1.pth'),
    #load_net('./ed5-mixup/best-fold-2.pth'),
    #load_net('../input/effdet5-folds-v2/fold3-best.bin'),
    #load_net('../input/effdet5-folds-v2/fold4-best.bin'),
#]
model = load_net('./ed5-mixup/best-fold-0.pth')

# Out of fold prediction:

In [9]:
metrics = eval_metrics(model, validation_loader)
metrics

{'best_score': 0.684,
 'best_threshold': 0.41,
 0.2: 0.5944,
 0.25: 0.6378,
 0.3: 0.663,
 0.4: 0.6836,
 0.45: 0.6829,
 0.5: 0.6758,
 0.55: 0.661,
 0.6: 0.6348}

In [9]:
for img, target, img_id in validation_loader:
    print(target)
    break

({'boxes': tensor([[ 44.5000, 128.0000, 101.0000, 181.5000],
        [108.0000, 141.0000, 170.5000, 200.5000],
        [ 98.5000,   0.0000, 139.5000,  17.0000],
        [252.0000, 279.0000, 352.0000, 321.0000],
        [209.5000, 180.5000, 278.0000, 229.0000],
        [192.0000, 121.5000, 238.5000, 163.0000],
        [377.0000,  92.0000, 419.0000, 150.0000],
        [164.5000, 205.0000, 214.5000, 247.5000],
        [218.5000,  20.5000, 256.0000,  61.5000],
        [169.0000, 438.5000, 277.0000, 481.0000],
        [129.0000,   0.0000, 175.0000,  38.0000],
        [137.0000,  31.5000, 194.5000,  83.0000],
        [437.5000, 389.0000, 512.0000, 425.5000],
        [354.0000, 293.5000, 399.0000, 333.5000],
        [ 64.0000, 181.5000, 107.5000, 218.5000],
        [405.5000, 480.0000, 486.0000, 512.0000],
        [359.5000,   0.0000, 403.5000,  31.5000],
        [200.5000,  72.5000, 250.5000, 114.5000],
        [264.5000,  44.5000, 309.5000,  82.0000],
        [ 38.0000, 375.0000, 115.0000, 